# [Advent of Code 2020](https://adventofcode.com/2020)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [13]:
import operator
import re
from functools import partial, reduce
from itertools import cycle, islice, starmap


def Input(day, line_parser=str.strip):
    "Fetch the data input from disk."
    filename = f'../data/advent2020/input{day}.txt'
    with open(filename) as fin:
        return mapt(line_parser, fin)

    
def mapt(fn, *args): 
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


def quantify(data, predictate=bool):
    "Count how many items in an iterable have predicated(item) True"
    return sum(map(predictate, data))


def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)

## [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

As usual, things start off easy enough, nested looping with a computationally simple upper bound. Finding numbers that add to 2020 and then returning their multiple.

In [14]:
def find_2020(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for n2 in nums[index:]:
            if n1 + n2 == 2020:
                return n1 * n2
    raise Exception('Unable to find solution')

In [15]:
data1 = Input(1, int)
find_2020(data1)

1006176

In [16]:
assert _ == 1006176, 'Day 1.1'

In [17]:
# part 2
def find_2020_3(nums: [int]) -> int:
    for index, n1 in enumerate(nums):
        for jindex, n2 in enumerate(nums[index+1:]):
            for n3 in nums[jindex+1:]:
                if n1 + n2 + n3 == 2020:
                    return n1 * n2 * n3
    raise Exception('Unable to find solution')

find_2020_3(data1)

199132160

In [18]:
assert _ == 199132160, 'Day 1.2'

# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

In [19]:
def is_valid_password(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    occurance = quantify(password, lambda c: c == match)
    return lower <= occurance <= upper

In [48]:
def password_and_match(line):
    chunks = re.sub('[-:]', ' ', line).strip().split(' ')
    lower, upper, match, _, password = chunks
    return password, (match, int(lower), int(upper))

data2 = Input(2, password_and_match)
quantify(starmap(is_valid_password, data2))

600

In [49]:
assert _ == 600, 'Day 2.1'

In [53]:
# part 2
def is_valid_password2(password: str, rule: (str, int, int)) -> bool:
    match, lower, upper = rule
    is_match = lambda n: password[n - 1] == match
    matches = int(is_match(lower)) + int(is_match(upper))
    return matches == 1

quantify(starmap(is_valid_password2, data2))

245

In [54]:
assert _ == 245, 'Day 2.2'

# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

Making use of `cycle` here means that account for the repeating behaviour of the grid. Equally we could have simply used some modular arithmitic but this is more fun (and execution time is trivial).

In [43]:
def count_trees(grid: [str], x_step: int, y_step: int):
    path = []
    for index, row in enumerate(grid[::y_step]):
        char_index = (x_step * index) % len(row)
        path.append(row[char_index])
    return quantify(path, lambda c: c == '#')
    

data3 = Input(3)
count_trees(data3, 3, 1)

151

In [44]:
assert _ == 151, 'Day 4.1'

In [45]:
# part 2
count_trees_3 = partial(count_trees, data3)
steps = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
reduce(
    operator.mul,
    starmap(count_trees_3, steps)
)

7540141059

In [40]:
assert _ == 7540141059, 'Day 3.2'